# DataFrames with Pandas and cuDF

## Table of Contents

1. [Introduction](#1.-Introduction)
2. [Pandas Essentials](#2.-Pandas-Essentials)
   - [2.1 Series and DataFrame Objects](#2.1-Series-and-DataFrame-Objects)
   - [2.2 Selecting and Filtering Data](#2.2-Selecting-and-Filtering-Data)
   - [2.3 Sorting](#2.3-Sorting)
   - [2.4 Summarizing Data](#2.4-Summarizing-Data)
   - [2.5 Grouped Aggregations (groupby)](#2.5-Grouped-Aggregations-(groupby))
   - [2.6 String Operations](#2.6-String-Operations)
   - [2.7 Time Series](#2.7-Time-Series)
   - [2.8 User-Defined Operations (apply)](#2.8-User-Defined-Operations-(apply))
3. [Enter cuDF: GPU DataFrames](#3.-Enter-cuDF:-GPU-DataFrames)
   - [3.1 Exercise: Date Formatting Failure](#3.1-Exercise:-Date-Formatting-Failure)
   - [3.2 Exercise: Why `.apply()` Breaks Down in cuDF](#3.2-Exercise:-Why-`.apply()`-Breaks-Down-in-cuDF)
4. [Exercise: Analyzing Real Data (NYC Parking Violations)](#4.-Exercise:-Analyzing-Real-Data-(NYC-Parking-Violations))
   - [Step 0: Download Data](#Step-0:-Download-Data)
   - [Task 1: Data Inspection (Pandas)](#📝-Task-1:-Data-Inspection-(Pandas))
   - [Task 2: Analyze Taxis (Pandas)](#📝-Task-2:-Analyze-Taxis-(Pandas))
   - [Task 3: GPU Acceleration (cuDF)](#📝-Task-3:-GPU-Acceleration-(cuDF))
5. [Conclusion](#Conclusion)

## 1. Introduction

In this notebook, we will build a foundation in data manipulation using **Pandas**, a popular tool for Python data analysis. Then, we will transition to **cuDF**, which allows us to run standard Pandas-like code on the GPU.


### Learning Objectives:

- **Introduce core Pandas operations:** Indexing, Filtering, Aggregating, and Time Series.
- **Learn the subtle differences** (and speed benefits) when porting code to cuDF.
- **Exercise:** Apply these skills to analyze a real-world NYC Parking Violations dataset.


**Note:** Throughout this notebook, we provide "Quick Docs" sections to remind you of common syntax. However, these are not exhaustive. For complete API details, parameters, and edge cases, you should always reference the official [Pandas Documentation](https://pandas.pydata.org/docs/) or the [cuDF Documentation](https://docs.rapids.ai/api/cudf/stable/).


---

## 2. Pandas Essentials

Before we accelerate with GPUs, let's ensure we are comfortable with the DataFrame API. Even if you are a Pandas pro, this refresher sets the baseline syntax we will replicate later.

First, import the library:


In [ ]:
import numpy as np
import pandas as pd

### 2.1 Series and DataFrame Objects

- **Series:** A one-dimensional labeled array (like a powerful list or a single column).
- **DataFrame:** A two-dimensional labeled data structure (like a spreadsheet or SQL table).

**Quick Docs:**

- `pd.Series(data)`: Create a Series.
- `pd.DataFrame(data, index)`: Create a DataFrame.
- `df.head(n)` / `df.tail(n)`: View the first/last n rows.
- `df.index` / `df.columns`: Access row labels and column names.


In [ ]:
# A Series acts like a single column of data
s = pd.Series([10, 20, 30])
print(f"Max value in series: {s.max()}")

# A DataFrame is a collection of Series sharing an index
df = pd.DataFrame({
    "a": [1, 2, 1, 3, 2],
    "b": [1, 4, 7, 2, 0],
    "c": [3, 3, 3, 4, 5]
}, index=[1, 2, 3, 4, 5])

# View the structure
print("Columns:", df.columns)
df.head(3)

### 2.2 Selecting and Filtering Data

Selecting specific subsets of data is the most common task in analysis. You can select by column name, label index, or integer position.

**Quick Docs:**

- `df['col']`: Select a single column (returns a Series).
- `df[['col1', 'col2']]`: Select multiple columns (returns a DataFrame).
- `df.loc[label]`: Select row(s) by index label.
- `df.iloc[position]`: Select row(s) by integer position (0-based).
- `df[condition]`: Boolean indexing (filtering).


In [ ]:
# Select specific columns
subset = df[["b", "c"]]

# Select rows by label (loc) and position (iloc)
row_label_2 = df.loc[2]     # Row with index label 2
row_pos_0 = df.iloc[0]      # First row (physically)

# Boolean Indexing: Filter rows where column 'a' is greater than 1
filtered_df = df[df['a'] > 1]
filtered_df

### 2.3 Sorting

Ordering data helps in ranking and visualization.

**Quick Docs:**

- `df.sort_values(by='col', ascending=True/False)`: Sort by one or more columns.


In [ ]:
# Sort by column 'a' in ascending order
sorted_df = df.sort_values("a")
sorted_df

### 2.4 Summarizing Data

It's straightforward to get a quick overview of your data's distribution.

**Quick Docs:**

- `df.describe()`: Summary statistics (count, mean, std, etc.).
- `df.mean()`, `df.sum()`, `df.max()`: Aggregations across columns.
- `df['col'].value_counts()`: Count unique values (useful for histograms).


In [ ]:
# Calculate the sum of every column
print("Sum of columns:\n", df.sum())

# Count frequency of values in column 'a'
print("\nValue counts for 'a':\n", df["a"].value_counts())

### 2.5 Grouped Aggregations (groupby)

The "Split-Apply-Combine" strategy. Split data into groups based on some criteria, apply a function to each group, and combine the results.

**Quick Docs:**

- `df.groupby('col')`: Group data.
- `.mean()`, `.count()`: Apply aggregation.
- `.agg({'col': ['min', 'max']})`: Apply complex, specific aggregations.


In [ ]:
# Group by 'a' and calculate the mean of 'b' and 'c' for each group
grouped_mean = df.groupby("a").mean()
print(grouped_mean)

# Complex aggregation: Get min and mean of 'b', and max of 'c'
agg_df = df.groupby("a").agg({
    "b": ["min", "mean"],
    "c": ["max"]
})
agg_df

### 2.6 String Operations

Pandas provides vectorized string functions via the `.str` accessor.

**Quick Docs:**

- `df['col'].str.upper()`: Convert to uppercase.
- `df['col'].str.contains('pattern')`: Boolean check for substring.
- `df['col'].str.replace('old', 'new')`: Replace text.

In [ ]:
# Add a string column
df["names"] = ["mario", "luigi", "yoshi", "peach", "toad"]

# Convert to uppercase
df["names_upper"] = df["names"].str.upper()
df

### 2.7 Time Series

Pandas was originally developed for financial time series analysis. It handles dates and times robustly via the `.dt` accessor.

**Quick Docs:**

- `pd.to_datetime()`: Convert strings to datetime objects.
- `df['date'].dt.year`: Extract year component.
- `df['date'].dt.dayofweek`: Extract day of week.


In [ ]:
# Create a date range
date_df = pd.DataFrame()
date_df["date"] = pd.date_range("2018-11-20", periods=5, freq="D")
date_df["value"] = np.random.sample(len(date_df))

# Filter by date
subset_dates = date_df[date_df["date"] < "2018-11-23"]

# Extract features
date_df["year"] = date_df["date"].dt.year
date_df

### 2.8 User-Defined Operations (apply)

When built-in functions aren't enough, you can apply custom Python functions.

**Quick Docs:**

- `df['col'].apply(func)`: Apply function `func` to every element.


In [ ]:
def add_ten(x):
    return x + 10

# Apply the custom function
df["a_plus_10"] = df["a"].apply(add_ten)
df.head(2)

---

## 3. Enter cuDF: GPU DataFrames

cuDF mimics the Pandas API but runs on the GPU. The transition is often as simple as changing the import, but there are some constraints you must know.

First, let's create a GPU DataFrame.


In [ ]:
import cudf

# Create a cuDF DataFrame (data resides on GPU)
gdf = cudf.DataFrame({
    "a": [1, 2, 1, 3, 2],
    "b": [1, 4, 7, 2, 0],
    "c": [1, 1, 8, 2, 9]
}, index=[1, 2, 3, 4, 5])

# Operations work exactly the same!
print(type(gdf))
gdf.groupby("a").mean()

### 3.1 Exercise: Date Formatting Failure

Pandas is very forgiving with date formats. cuDF is stricter. Run the cell below to see what happens when you use a non-standard date string.


In [ ]:
# EXECUTE THIS CELL TO SEE THE ERROR
try:
    date_df = cudf.DataFrame()
    # Pandas handles "11/20/2018" easily. Does cuDF?
    date_df["date"] = cudf.date_range("11/20/2018", periods=72, freq="D")
except Exception as e:
    print(f"Error caught: {e}")

**Why did this fail?** Unlike Pandas, cuDF currently requires ISO-standard date formats (Year-Month-Day) for creating date ranges.

- **Pandas:** Guesses `11/20/2018` is Nov 20th.
- **cuDF:** Requires `2018-11-20`.


### 3.2 Exercise: Why `.apply()` Breaks Down in cuDF

In Pandas, `.apply()` works by executing Python-level code over the data, which makes it very flexible but CPU-centric. On the GPU, this model does not translate directly: GPUs cannot execute arbitrary Python bytecode. In cuDF, `.apply()`-style user-defined functions must be JIT-compiled with Numba for the CUDA target, which imposes constraints:
- The function must be Numba-compilable (no general Python objects or unsupported features).
- Only operations that Numba can lower to GPU device code are allowed.
- Code must follow a more restricted, compilation-friendly execution model.

Note that cuDF UDFs are not limited to pure math. Some string operations and well-defined null-handling patterns are supported. See the [cuDF apply() documentation](https://docs.rapids.ai/api/cudf/latest/user_guide/api_docs/api/cudf.dataframe.apply/) for the full set of supported features and limitations.

Even simple-looking Python functions often fall outside these constraints:

In [ ]:
# A function that looks innocent but is NOT GPU-safe
def add_ten_verbose(x):
    # Python branching + dynamic typing make this un-compilable for the GPU
    if isinstance(x, (int, float)):
        return x + 10
    else:
        return x

This is perfectly legal in Pandas. But in cuDF, Numba cannot:

- interpret `isinstance`
- handle Python branching on object types
- JIT-compile dynamic return values

Now try running it:


In [ ]:
# Execute this cell to observe the cuDF limitation
try:
    gdf["a"] = gdf["a"].apply(add_ten_verbose)
except Exception as e:
    print("cuDF apply() constraint caught:")
    print(e)

**What happens?** cuDF attempts to compile the function → compilation fails → you get a runtime error. This mirrors real-world failure modes: anything that is not pure numerical logic will break.

Here is the same logic, rewritten in a way the GPU can compile:

In [ ]:
# GPU-safe version: no Python, no branching, pure math
def add_ten_gpu(x):
    return x + 10
try:
    gdf["a"] = gdf["a"].apply(add_ten_gpu)
except Exception as e:
    print("cuDF apply() constraint caught:")
    print(e)

This is one of the few forms that Numba can translate. But even this version is not preferred. **cuDF `.apply()` is a last resort**. Even if your function compiles, `.apply()` still triggers:

- JIT compilation overhead (slow startup)
- Kernel launch overhead
- Reduced optimization compared to built-in GPU operations

For typical column transformations, this is simply unnecessary. 
**Best practice is to always use vectorized operations:**

```python
gdf["a"] + 10
```
The vectorized version is:
- faster
- simpler
- more readable


---

## 4. Exercise: Analyzing Real Data (NYC Parking Violations)

Now you will apply what you learned to a large, real-world dataset.


### Step 0: Download Data

We will fetch a subset of the NYC Parking Violations dataset (Fiscal Year 2022).


In [ ]:
!wget -nc https://data.rapids.ai/datasets/nyc_parking/nyc_parking_violations_2022.parquet -O nyc_parking_violations_2022.parquet

### Task 1: Data Inspection (Pandas)

**Goal:** Load the data and inspect its structure.

**Instructions:**

1. Read the file `nyc_parking_violations_2022.parquet` into a Pandas DataFrame.
2. Print the columns.
3. Create a subset DataFrame with only: `Registration State`, `Violation Description`, `Vehicle Body Type`, `Issue Date`.
4. Display the head of this subset.


In [ ]:
import pandas as pd

# Read parquet file
df = pd.read_parquet("nyc_parking_violations_2022.parquet")

# Print columns
print(df.columns)

# Select specific columns
df_subset = df[["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date"]]

# Display head
df_subset.head()

### Task 2: Analyze Taxis (Pandas)

**Goal:** Filter, Group, and Count.

**Instructions:**

1. Filter the DataFrame to find rows where `Vehicle Body Type` is `"TAXI"`.
2. Group by `Registration State`.
3. Count the occurrences to see which states the taxis are registered in.
4. Sort the results descending to find the top states.


In [ ]:
%%time

# Filter for TAXI
taxi_df = df_subset[df_subset["Vehicle Body Type"] == "TAXI"]

# Group by State and count
state_counts = taxi_df.groupby("Registration State").size()

# Sort and display top results
state_counts.sort_values(ascending=False)

### Task 3: GPU Acceleration (cuDF)

**Goal:** Measure the speedup.

**Instructions:**

1. Import `cudf`.
2. Use `%%time` at the top of the cell.
3. Replicate the entire pipeline (Read -> Filter columns -> Filter Rows -> Group -> Sort) using `cudf`.


In [ ]:
%%time
import cudf

# Read parquet file into cuDF DataFrame
gdf = cudf.read_parquet("nyc_parking_violations_2022.parquet")

# Select specific columns (Filter columns)
gdf_subset = gdf[["Registration State", "Violation Description", "Vehicle Body Type", "Issue Date"]]

# Filter for TAXI (Filter Rows)
taxi_gdf = gdf_subset[gdf_subset["Vehicle Body Type"] == "TAXI"]

# Group by State and count
state_counts_gpu = taxi_gdf.groupby("Registration State").size()

# Sort and display top results
state_counts_gpu.sort_values(ascending=False)

---

## Conclusion

Compare the **Wall time** of Task 2 vs Task 3. You should see a significant performance improvement with cuDF, especially as data size grows!
